# Buffered H3 Polygon Demo

This notebook demonstrates `get_buffered_h3_polygon` - a fast alternative to computing boundary children.

In [1]:
from h3_toolkit import cpp_geom_available
print(cpp_geom_available())  # True/False

True


In [2]:
#from h3_toolkit import get_buffered_boundary_polygon_cpp
#coords = get_buffered_boundary_polygon_cpp(cell, intermediate_res=10, buffer_meters=-1.0)
# coords is list of (lon, lat) tuples

In [3]:
import h3
#from h3_toolkit import get_buffered_boundary_polygon
#from h3_toolkit._h3_toolkit_cpp import get_buffered_boundary_polygon
#from h3_toolkit import get_buffered_h3_polygon, get_buffered_boundary_polygon
from h3_toolkit import get_buffered_boundary_polygon_cpp, cell_boundary_from_children_cpp

import folium
import time
from h3_toolkit import (
    get_buffered_h3_polygon, 
    cell_boundary_to_geojson, 
    cell_boundary_from_children,
    get_backend
)

print(f"Backend: {get_backend()}")

Backend: cpp


## 1. Performance Comparison

In [11]:
cell = h3.latlng_to_cell(37.7749, -122.4194, 2)
print(f"Cell: {cell} (Resolution {h3.get_resolution(cell)})")

# Buffered polygon (fast)
start = time.time()
#buffered = get_buffered_h3_polygon(cell)
# Get boundary at res 10, then buffer it
buffered = get_buffered_boundary_polygon_cpp(cell, intermediate_res=7, use_convex_hull=True)
# Or with custom buffer
#buffered = get_buffered_boundary_polygon(cell, intermediate_res=10, buffer_meters=50)
#result = get_buffered_boundary_polygon(cell, 10, -1.0)  # 100% edge buffer

print(f"\nget_buffered_h3_polygon: {(time.time() - start)*1000:.2f} ms")
print(f"  Buffer: {buffered['properties']['buffer_meters']:.2f} meters")

# Boundary from children (slower, more accurate)
start = time.time()
boundary = cell_boundary_from_children_cpp(cell, 9)
print(f"\ncell_boundary_from_children(res=13): {(time.time() - start)*1000:.2f} ms")
print(f"  Boundary cells: {boundary['properties']['num_boundary_cells']}")

Cell: 822837fffffffff (Resolution 2)

get_buffered_h3_polygon: 2.87 ms
  Buffer: 1406.48 meters

cell_boundary_from_children(res=13): 12619.04 ms
  Boundary cells: 6558


## 2. Compare: Original vs Buffered vs Boundary from Children

In [12]:
lat, lng = h3.cell_to_latlng(cell)
m = folium.Map(location=[lat, lng], zoom_start=14)

# Boundary from children (blue - most accurate)
folium.GeoJson(
    boundary,
    style_function=lambda x: {'color': 'blue', 'fillOpacity': 0.1, 'weight': 2},
    name='Boundary from Children (accurate)'
).add_to(m)

# Original cell (green dashed)
folium.GeoJson(
    cell_boundary_to_geojson(cell),
    style_function=lambda x: {'color': 'green', 'fillOpacity': 0, 'weight': 2, 'dashArray': '10,5'},
    name='Original Cell'
).add_to(m)

# Buffered polygon (red dashed)
folium.GeoJson(
    buffered,
    style_function=lambda x: {'color': 'red', 'fillOpacity': 0.05, 'weight': 2, 'dashArray': '5,5'},
    name='Buffered (fast approximation)'
).add_to(m)

folium.LayerControl().add_to(m)
m

## 3. Performance Table

Comparing `get_buffered_h3_polygon` (fast) vs `cell_boundary_from_children` (accurate)

In [6]:
print("| Res | Buffered (ms) | Boundary from Children (ms) | Boundary Cells |")
print("|-----|---------------|-----------------------------|--------------------|")

for res in range(6, 12):
    cell = h3.latlng_to_cell(37.7749, -122.4194, res)
    target = min(res + 5, 15)  # Max target res 15
    
    start = time.time()
    buf = get_buffered_h3_polygon(cell)
    buf_time = (time.time() - start) * 1000
    
    start = time.time()
    bound = cell_boundary_from_children(cell, target)
    bound_time = (time.time() - start) * 1000
    
    n_cells = bound['properties']['num_boundary_cells']
    print(f"| {res:3} | {buf_time:13.2f} | {bound_time:27.2f} | {n_cells:18} |")

| Res | Buffered (ms) | Boundary from Children (ms) | Boundary Cells |
|-----|---------------|-----------------------------|--------------------|
|   6 |          1.74 |                       34.49 |                726 |
|   7 |          0.55 |                       88.09 |                726 |
|   8 |          0.67 |                       93.99 |                726 |
|   9 |          0.64 |                       88.61 |                726 |
|  10 |          0.37 |                       90.89 |                726 |
|  11 |          0.66 |                       18.30 |                240 |
